In [10]:
import os
import sys
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import warnings
import pandas as pd
from ciceroscm import input_handler
import logging
logging.disable(logging.CRITICAL)



In [11]:
sys.path.insert(0,os.path.join(os.getcwd(), '../'))
module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)


In [12]:
from meteor import MeteorPatternScaling, meteor_plot_utils

In [13]:
from meteor.prpatt import global_mean
from meteor.scm_forcer_engine import ScmEngineForPatternScaling, run_single_experiment

In [14]:
from dataclasses import asdict

In [15]:
data_dir = os.path.join('/div/no-backup/users/masan/temp/')


In [16]:
mdls=['CanESM2','GISS-E2-R','NorESM1','MIROC5']

In [17]:
exp_list=["base", "co2x2","sulx5"]

In [9]:
# NBVAL_IGNORE_OUTPUT
patt={}
for m in mdls:
    patt[m]= MeteorPatternScaling(
        "pdrmip-"+m,
        {"tas": 2, "pr": 2},
        lambda exp: os.path.join(data_dir, f"pdrmip-{exp}_T42_ANN_"+m+".nc"),
        exp_list=exp_list,
        )



/home/masan/gitrepos/METEOR/venv/lib/python3.9/site-packages/ciceroscm/input_handler.py:98: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_input = pd.read_csv(
/home/masan/gitrepos/METEOR/venv/lib/python3.9/site-packages/ciceroscm/input_handler.py:98: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_input = pd.read_csv(
/home/masan/gitrepos/METEOR/venv/lib/python3.9/site-packages/ciceroscm/input_handler.py:39: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df_gas = pd.read_csv(filename, delim_whitespace=True, index_col=0)
/home/masan/gitrepos/METEOR/src/meteor/scm_forcer_engine.py:99: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chaine

11
12
113
114
115
22
141
123
142
11
12
113
114
115
22
141
123
142
11
12
113
114
115
22
141
123
142


FileNotFoundError: [Errno 2] No such file or directory: '/div/no-backup/users/masan/temp/pdrmip-base_T42_ANN_CanESM2.nc'

Plot reconstructions of the pulse-response function

In [ ]:
# NBVAL_IGNORE_OUTPUT
f, axs=plt.subplots(len(mdls),4,figsize=[15,15])
for i,m in enumerate(mdls):
    meteor_plot_utils.plot_global_mean_values(patt[m], axs[i,0],'tas','co2x2')
    meteor_plot_utils.plot_global_mean_values(patt[m], axs[i,1],'pr','co2x2')
    meteor_plot_utils.plot_global_mean_values(patt[m], axs[i,2],'tas','sulx5')
    meteor_plot_utils.plot_global_mean_values(patt[m], axs[i,3],'pr','sulx5')
plt.tight_layout()

In [ ]:
# NBVAL_IGNORE_OUTPUT
#plot spatial patterns associated with each mode
f, ax = plt.subplots(2,2)
ax[0,0].pcolor(patt[mdls[0]].pattern_dict['co2x2']['tas']["orgeof"]["v"][0, :, :],cmap="bwr",vmin=-2,vmax=2)
ax[0,0].set_title('Mode 1 - EOF')

ax[1,0].pcolor(patt[mdls[0]].pattern_dict['co2x2']['tas']["orgeof"]["v"][1, :, :],cmap="bwr",vmin=-2,vmax=2)
ax[1,0].set_title('Mode 2 - EOF')

ax[0,1].plot(patt[mdls[0]].pattern_dict['co2x2']['tas']["orgeof"]["u"][:,0],label='original')
ax[0,1].plot(patt[mdls[0]].pattern_dict['co2x2']['tas']["neweof"]["u"][:,0],label='fit')
ax[0,1].set_title('Mode 1 - PC')
ax[0,1].legend()
ax[1,1].plot(patt[mdls[0]].pattern_dict['co2x2']['tas']["orgeof"]["u"][:,1],label='original')
ax[1,1].plot(patt[mdls[0]].pattern_dict['co2x2']['tas']["neweof"]["u"][:,1],label='fit')
ax[1,1].set_title('Mode 2 - PC')
ax[1,1].legend()


Model test

In [ ]:
#define a gaussian forcing profile

f=1*np.exp(-np.square(np.arange(0,1000)-150)/5000)



In [ ]:
#run the emulator

mdl_out=patt[mdls[0]].predict_from_forcing_profile(f, 'tas', exp="co2x2")

In [ ]:
# NBVAL_IGNORE_OUTPUT
#plot global mean forcing and temperature to illustrate lagged response
gmt=global_mean(mdl_out)

fig, ax1 = plt.subplots()

color = 'tab:red'
ax1.set_xlabel('year')
ax1.set_ylabel('$Wm^{-2}$', color=color)
ax1.plot(f, color=color)
ax1.tick_params(axis='y', labelcolor=color)

ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
ax2.set_ylabel('GMTS', color=color)  # we already handled the x-label with ax1
ax2.plot(gmt, color=color)
ax2.axhline(0)
ax2.tick_params(axis='y', labelcolor=color)

fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()


In [ ]:
def predict_per_experiment(pattern, experiment):
    conc_data = input_handler.read_inputfile(
    os.path.join("/div/amoc/CSCM/SCM_Linux_v2019/RCMIP/input", f"{experiment}_conc_RCMIP.txt")
)
    ih = input_handler.InputHandler({})
    em_data = ih.read_emissions(os.path.join("../tests/test-data", f"{experiment}_em_RCMIP.txt"))
    patterns = pattern.predict_from_combined_experiment(em_data, conc_data, ["pr", "tas"])
    cfg = {
            "conc_run": False,
            "nystart": em_data.index[0],
            "emstart": em_data.index[0] + 100,
            "nyend": 2100,
            "concentrations_data": conc_data,
            "emissions_data": em_data,
            }
    sefps = ScmEngineForPatternScaling(cfg)
    fcg = sefps.run_and_return_per_forcer_results(pattern.exp_list)
    fcg_ttl = run_single_experiment(asdict(sefps.cfg), sefps.input_h)

    return patterns,fcg,fcg_ttl,em_data,conc_data


In [ ]:


def make_pattern(name, exp_list):
    
    pattern = MeteorPatternScaling(name, {"tas":2, "pr":10}, lambda exp: os.path.join("/div/no-backup/users/masan/temp", f"pdrmip-{exp}_T42_ANN_NorESM1.nc"), exp_list = exp_list)
    return pattern

In [ ]:

experiments = ["rcp85", "rcp60", "rcp45", "rcp26"]
exp_lists = [["base","co2x2"], ["base","co2x2", "bcx10"],  ["base","co2x2", "sulx5"], ["base","co2x2", "bcx10", "sulx5"]] # ,
components = ["tas", "pr"]


In [ ]:
exp="rcp85"
comp="pr"
p=make_pattern("pdrmip-NorESM1-all", exp_lists[3])
patterns,fcg,fttl,em,conc = predict_per_experiment(p,exp) 


In [ ]:
data_hist = xr.open_dataset(f"/div/no-backup/users/masan/temp/{comp}_yr_NorESM1_historical_r1i1p1_185001-200512.nc")[comp]
data_rcp = xr.open_dataset(f"/div/no-backup/users/masan/temp/{comp}_yr_NorESM1_{exp}_r1i1p1_200601-210012.nc")[comp]
hist_ts = global_mean(data_hist)
tot_ts = xr.concat((hist_ts - hist_ts[0], global_mean(data_rcp) - hist_ts[0]),dim='time')
tot_ts=tot_ts.convert_calendar("Gregorian")


In [ ]:
tot_ts.plot()
global_mean(patterns[comp]).plot()
plt.ylim(min(tot_ts),max(tot_ts))


In [ ]:
fcgarr=np.zeros((len(fttl.keys()),len(patterns[comp]['time'].values)))
lbl=[]
print(fcgarr.shape)
for i,f in enumerate(fttl.keys()):
    fcgarr[i,:]=fttl[f]
    lbl.append(f)

In [ ]:
plt.plot(patterns[comp]['time'],fttl['Total_forcing'])
plt.plot(patterns[comp]['time'],fttl['CO2'])


In [ ]:
plt.stackplot(patterns[comp]['time'],fcgarr[:-1,:],labels=lbl[:-1])
plt.legend()


In [ ]:
plt.plot(fttl['N2O'])

In [ ]:
em['N2O'].plot()

In [ ]:
plt.plot(patterns[comp]['time'],fttl['Total_forcing'])

In [ ]:
plt.plot(patterns[comp]['time'],fcg['co2x2'])